In [6]:
from bs4 import BeautifulSoup
import urllib.request as url 
import requests
import pprint
import re
from pandas import DataFrame


In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def collectData(file):
    
    result_name=[]
    result_symptoms = []
    result_causes = []
    result_comments = []
    result_managements = []
    
    #reading urls from .xlsx sheet
    df = pd.read_excel(file)
    links = df['links']
    for x in links:
        try:
            #stripping the last part of url for file name
            text = ((x.rsplit('/', 2)[1]).rsplit('.',1)[0])
            file_name = text + '.xlsx'
            page = requests.get(x)
            soup = BeautifulSoup(page.text, 'html.parser')
            names = soup.find_all('h4')
            
            for name in names:
                if name.text.strip().encode('utf8') != '':
                    name_after_splitted = name.text.strip()
                    result_name.append(name_after_splitted)
            prefixes = ('Category','Caused')
            Name = result_name
            for word in Name[:]:
                if word.startswith(prefixes):
                    Name.remove(word)

            import re
            sep = '\n'
            lines = []
            for line in Name:
                res = line.split(sep, 1)[0]
                res1 = text + '_' + res
                lines.append(res1)
            
            symptoms = soup.find_all('div', class_='symptoms')
            causes = soup.find_all('div', class_='cause')
            comments = soup.find_all('div', class_='comments')
            managements = soup.find_all('div', class_='management')
            # print symptoms
            for symptom in symptoms:
                if symptom.text.strip().encode('utf8') != '':
                    symptom_after_splitted = symptom.text.strip()
                    result_symptoms.append(symptom_after_splitted)
            for cause in causes:
                if cause.text.strip().encode('utf8') != '':
                    cause_after_splitted = cause.text.strip()
                    result_causes.append(cause_after_splitted)
            for comment in comments:
                if comment.text.strip().encode('utf8') != '':
                    comment_after_splitted = comment.text.strip()
                    result_comments.append(comment_after_splitted)
              # print managements
            for management in managements:
                if management.text.strip().encode('utf8') != '':
                    management_after_splitted = management.text.strip()
                    result_managements.append(management_after_splitted[23:])
            from pandas import DataFrame
            df['Name'] = DataFrame (lines,columns=['Name'])
            df['Symptoms'] = pd.DataFrame(result_symptoms,columns=['Symptoms'])
            df['Causes'] = pd.DataFrame(result_causes,columns=['Causes'])
            df['Comments'] = pd.DataFrame(result_comments,columns=['Comments'])
            df['Management'] = pd.DataFrame( result_managements,columns=['Management'])
            #exporting all questions to .xlsx sheet
            df.to_excel(file_name,index=False,encoding='utf-8')
        except:
            print("There was a problem with the link: ", x)

In [8]:
collectData ('F:/plantvillage.xlsx')